In [1]:
import pandas as pd
from pandas import merge
import ast
import numpy as np
import gc
#import tqdm
from tqdm._tqdm_notebook import tqdm_notebook as tq

In [2]:
movie_data = pd.read_csv('test.csv')

In [3]:
movie_data.shape

(1647, 8)

In [3]:
ones = [1]*movie_data.shape[0]
movie_data['ones'] = ones
del ones

In [5]:
cross_join = merge(movie_data[['movie id','director','creator','actor_list','genre','ones']],movie_data[['movie id','director','creator','actor_list','genre','ones']],on = "ones")

In [4]:
def calc_similarity(row,l = 0.25):
    #gc.collect()
    features = ['director','creator','actor_list','genre']
    feature_vector = []
    for feature in features:
        fx = row[feature+'_x']
        fy = row[feature+'_y']
        if isinstance(fx,str):
            fx = fx.replace('“','"')
            fx = fx.replace('”','"')
            fx = ast.literal_eval(fx)
        if isinstance(fy,str):
            fy = fy.replace('“','"')
            fy = fy.replace('”','"')
            fy = ast.literal_eval(fy)
        fx = set(fx)
        fy = set(fy)
        a = len(fx.intersection(fy))
        b = len(fx.union(fy))
        feature_vector.append(float(a)/float(b))
    feature_vector = np.array(feature_vector)
    weight_vector = np.array([l]*len(feature_vector))
    similarity_vector = feature_vector.dot(weight_vector)
    row['similarity'] = similarity_vector
    del features,feature_vector,weight_vector,a,b,similarity_vector
    return row[['movie id_x','movie id_y','similarity']]

In [9]:
sample_cross_join = merge(movie_data.loc[0:20,['movie id','director','creator','actor_list','genre','ones']],movie_data.loc[:,['movie id','director','creator','actor_list','genre','ones']],on = "ones")
tq().pandas(desc = "my bar")

26458it [01:28, 360.50it/s]

In [10]:
similarity_matrix = sample_cross_join.progress_apply(calc_similarity,axis = 1)

In [12]:
similarity_matrix.shape

(34587, 3)

In [51]:
type(tq())

tqdm._tqdm_notebook.tqdm_notebook

2230it [00:21, 278.74it/s]

In [13]:
similarity_matrix.to_csv('similarity.csv')

In [5]:
for i in xrange(0,1647,61):
    sample_cross_join = merge(movie_data.loc[i:i+60,['movie id','director','creator','actor_list','genre','ones']],movie_data.loc[:,['movie id','director','creator','actor_list','genre','ones']],on = "ones")
    tq().pandas(desc = "my bar")
    similarity_matrix = sample_cross_join.progress_apply(calc_similarity,axis = 1)
    similarity_matrix.to_csv('similarity.csv',mode = 'a')
    del sample_cross_join,similarity_matrix

100468it [04:50, 352.21it/s]


100468it [04:30, 365.93it/s]


In [6]:
similarity_matrix = pd.read_csv('similarity.csv')
similarity_matrix.shape

/home/jatin/anaconda2/envs/gl-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2712635, 4)

In [8]:
movie_data

,_id,movie id,movie title,director,creator,actor_list,genre,release_date,ones
0,tt0114709,1,Toy Story (1995),"[""John Lasseter""]","[""John Lasseter"",""Pete Docter"",""Pixar Animatio...","[""Tom Hanks"",""Tim Allen"",""Don Rickles"",""Jim Va...","["" Animation"","" Adventure"","" Comedy"","" Family""...",1995-11-22,1
1,tt0113189,2,GoldenEye (1995),"[""Martin Campbell""]","[""Ian Fleming"",""Michael France"",""Eon Productio...","[""Pierce Brosnan"",""Sean Bean"",""Izabella Scorup...","["" Action"","" Adventure"","" Thriller""]",1995-11-17,1
2,tt0113101,3,Four Rooms (1995),"[""Allison Anders"",""Alexandre Rockwell""]","[""Allison Anders"",""Alexandre Rockwell"",""Mirama...","[""Sammi Davis"",""Amanda De Cadenet"",""Valeria Go...","["" Comedy""]",1995-12-25,1
3,tt0113161,4,Get Shorty (1995),"[""Barry Sonnenfeld""]","[""Elmore Leonard"",""Scott Frank"",""Metro-Goldwyn...","[""John Travolta"",""Gene Hackman"",""Rene Russo"",""...","["" Comedy"","" Crime"","" Thriller""]",1995-10-20,1
4,tt0112722,5,Copycat (1995),"[""Jon Amiel""]","[""Ann Biderman"",""David Madsen"",""Regency Enterp...","[""Sigourney Weaver"",""Holly Hunter"",""Dermot Mul...","["" Crime"","" Drama"","" Mystery"","" Thriller""]",1995-10-27,1
5,tt0115012,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,"[""Yimou Zhang""]","[""Bi Feiyu"",""Li Xiao"",""Alpha Films"",""La Sept C...","[""Li Gong"",""Baotian Li"",""Xiaoxiao Wang"",""Xueji...","["" Crime"","" Drama"","" Romance"","" Thriller""]",1995-11-08,1
6,tt0114746,7,Twelve Monkeys (1995),"[""Terry Gilliam""]","[""Chris Marker"",""David Webb Peoples"",""Universa...","[""Joseph Melito"",""Bruce Willis"",""Jon Seda"",""Mi...","["" Mystery"","" Sci-Fi"","" Thriller""]",1996-01-05,1
7,tt0112431,8,Babe (1995),"[""Chris Noonan""]","[""Dick King-Smith"",""George Miller"",""Universal ...","[""Christine Cavanaugh"",""Miriam Margolyes"",""Dan...","["" Comedy"","" Drama"","" Family""]",1995-08-04,1
8,tt0112818,9,Dead Man Walking (1995),"[""Tim Robbins""]","[""Helen Prejean"",""Tim Robbins"",""Havoc"",""Polygr...","[""Susan Sarandon"",""Sean Penn"",""Robert Prosky"",...","["" Crime"","" Drama""]",1996-02-02,1
9,tt0114279,10,Richard III (1995),"[""Richard Loncraine""]","[""William Shakespeare"",""Ian McKellen"",""Mayfair...","[""Ian McKellen"",""Annette Bening"",""Jim Broadben...","["" Drama"","" War""]",1995-12-29,1
